In [1]:
!pip3 install python-dateutil lxml requests selectorlib transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import csv
from dateutil import parser as dateparser

# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('/content/selectors.yml')

def scrape(url):    
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open("urls.txt",'r') as urllist, open('data.csv','w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=["title","content","date","variant","images","verified","author","rating","product","url"],quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for url in urllist.readlines():
        data = scrape(url) 
        if data:
            for r in data['reviews']:
                r["product"] = data["product_title"]
                r['url'] = url
                if 'verified' in r:
                    if 'Verified Purchase' in r['verified']:
                        r['verified'] = 'Yes'
                    else:
                        r['verified'] = 'Yes'
                r['rating'] = r['rating'].split(' out of')[0]
                date_posted = r['date'].split('on ')[-1]
                if r['images']:
                    r['images'] = "\n".join(r['images'])
                r['date'] = dateparser.parse(date_posted).strftime('%d %b %Y')
                writer.writerow(r)
            # sleep(5)

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('data.csv')  

In [4]:
df

,title,content,date,variant,images,verified,author,rating,product,url
0,Perfect Tiny Living appliance,I use this for everything! How did I live with...,08 Oct 2017,Size: 3Qt Pattern Name: Pressure Cooker,https://images-na.ssl-images-amazon.com/images...,Yes,The Galavan,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
1,Wish I got the 6QT instead of the 3QT,UPDATED - So I finally got the 6QT and oh. my....,24 Apr 2018,Size: 3Qt Pattern Name: Pressure Cooker,NaN,Yes,DLO,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
2,Didn't Realize How Much I Needed the Mini !!,"I've been using the 6 qt IP, but always had wa...",13 Sep 2017,Size: 3Qt Pattern Name: Pressure Cooker,NaN,Yes,Royal2,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
3,Bad customer service!,"I was loving my instant pot, bragging to all a...",29 Aug 2018,Size: 8Qt Pattern Name: Pressure Cooker,NaN,Yes,HomeForSummer,1.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
4,DANGER,"Im not sure what happened here, because i know...",03 Dec 2018,Size: 8Qt Pattern Name: Pressure Cooker,https://images-na.ssl-images-amazon.com/images...,Yes,marvin gonzalez,1.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
...,...,...,...,...,...,...,...,...,...,...
95,thinking about buying?/ you will not be sorry,I bought this to make things easier. It made ...,11 Nov 2017,Size: 3Qt Pattern Name: Pressure Cooker,NaN,Yes,Jefff,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
96,Almost the same as my previous IP,My old IP died after 6 years of frequent use a...,04 Nov 2020,Size: 6Qt Pattern Name: Pressure Cooker,NaN,Yes,David and Shoshana Cooper,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
97,If I could only own one kitchen appliance this...,I really love this pot. I will probably buy a...,27 Oct 2017,Size: 3Qt Pattern Name: Pressure Cooker,NaN,Yes,R. Jones,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...
98,Easy to use and quiet,I love love love this Instant Pot. Everything...,11 Aug 2020,Size: 6Qt Pattern Name: Pressure Cooker,NaN,Yes,JustFlo,5.0,Instant Pot Duo 7-in-1 Electric Pressure Cooke...,https://www.amazon.com/Instant-Pot-Multi-Use-P...


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [6]:
import torch
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [7]:
df2 = pd.DataFrame(df['content'])

In [8]:
df2.head()

,content
0,I use this for everything! How did I live with...
1,UPDATED - So I finally got the 6QT and oh. my....
2,"I've been using the 6 qt IP, but always had wa..."
3,"I was loving my instant pot, bragging to all a..."
4,"Im not sure what happened here, because i know..."


In [9]:
df2['sentiment'] = df['content'].apply(lambda x: sentiment_score(x[:512]))

In [10]:
pd.set_option('display.max_colwidth', None)
df2.head()

,content,sentiment
0,"I use this for everything! How did I live without this before??? I live in a Sprinter van full-time and was using an induction burner for cooking, but since I've gotten this little gem, I haven't used the induction burner once! This is so easy to use, and it's the perfect size for one or two people. I've been making a lot of stews, and I'll saute the onions, carrots, and meat (if I'm using meat) then add everything else, seal it, and set it for 7 minutes on high pressure. I let it release naturally and have a perfectly cooked, tasty meal every time. This gives me enough for 4 or 5 servings. I also quick soaked beans the other day by covering them with water, hitting pressure for 5 min. then letting it release naturally and let it sit (not on keep warm) for a couple of hours. I'll often make dinner right after I reheat my lunch, and make sure the keep warm function is on. I'm able to drive with it in my sink and have a hot, healthy, delicious meal waiting for me when I'm done driving. I've even baked banana bread in it!!! So exciting, because I don't have an oven. It turned out SO good! I used one of the containers and the lid from my To-Go Ware stainless steel lunch container to bake it in on the rack. Because this cooks so quickly, it is very easy on my solar system. I can reheat leftovers on saute in 4 minutes, and it's only drawing 67a while it's on. I did have a VitaClay, that I absolutely loved for making beans, and especially bone broth, but it takes much longer to cook and is heavier. So, more battery drain, more storage space, and more weight as compared to the Instant Pot Mini and the Mini wins on each count!",5
1,"UPDATED - So I finally got the 6QT and oh. my. g... - so wonderful! Such a relief! I can cook 5 lbs of frozen chicken in ONE cycle and have food for my dog for the next week! Yay! And last night I finally made that famous ""soccer mom spaghetti"" (dump a box of pasta, one jar of sauce, a bag of frozen meatballs plus a jar of water in the pot then push a button) and - perfect. I made this same thing in the 3qt about a few times and it never came out right - first the pasta didn't cook, next time the sauce was too watery, etc. With the 6QT - cooked perfectly! And I have FIVE portions of leftovers instead of the 1-2 I had with the smaller I.P. To put the 3qt vs 6qt in perspective: the 3qt pot does not actually cook 3 quarts because you can't fill it all the way to the top. There's a line that's oh, an inch or two below the lip that's the Max Fill line, because the pressure cooker needs extra room to do its thing. Then, if you're adding the trivet, you're losing another inch of space. So really your cooking space is more like 2 quarts, like a small sauce pan. Imagine you're cooking ALL of your meals with nothing bigger than a sauce pan. Then you've got the 6qt pot, which is like the big pots you use to boil pasta. Or make soup. Pasta pot vs sauce pan. You see what I'm saying? Anyway, now my problem is that I have to stop cooking so often because I have no more room in my freezer... :) ******** ABOUT THE 3 QT DUO --- As Instant Pots go, I'll not reiterate what everyone else says but agree that it's a game changer, and totally revolutionizes your cooking. However, I have some regrets... 3QT Too Small for Lazy Cooks Like Me --- When I to move into my Airstream trailer full time, I decided to get an Instant Pot as I thought it would help minimize the amount of cooking appliances I needed. And of course, being solo (plus a dog and cat) I figured the 3Qt was the better choice. After 7 months with it, I am desperate to trade my 3Qt for the 6QT. I like cooking, but I'm lazy and would prefer to make a large batch once and have left overs, than have to cook for every meal. Reheating leftovers is far easier to clean up than cooking a fresh meal. I'm lazy at cleaning, too. Instant Pot Learning Curve --- Instant Pot cooking takes some getting used to, and while there is a great abundance of tips and su